In [2]:
!pwd

/rhome/eingerman/Projects/DeepLearning/TTS/Kokoro-LiteRT/kokoro_litert


In [3]:
import tensorflow as tf
from ai_edge_litert.interpreter import Interpreter
from kokoro.model import KModelTF

/rhome/eingerman/Projects/DeepLearning/TTS/Kokoro-LiteRT/.venv/lib/python3.12/site-packages/tensorflow_hub/__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [5]:
model = tf.keras.models.load_model('../kokoro.keras', custom_objects={
    'KModelTF': KModelTF,
})

TypeError: <class 'kokoro.model.KModelTF'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'kokoro_litert.kokoro.model', 'class_name': 'KModelTF', 'config': {'name': 'k_model_tf', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}}, 'registered_name': 'Custom>KModelTF', 'build_config': {'input_shape': {'input_ids': [1, None], 'ref_s': [1, 256], 'speed': 1}}, 'compile_config': {}}.

Exception encountered: Unable to revive model from config. When overriding the `get_config()` method, make sure that the returned config contains all items used as arguments in the  constructor to <class 'kokoro.model.KModelTF'>, which is the default behavior. You can override this default behavior by defining a `from_config(cls, config)` class

In [3]:
interpreter = Interpreter(model_path="../kokoro.tflite")

signatures = interpreter.get_signature_list()
print(signatures)
print(interpreter.get_input_details())

{}
[{'name': 'input_ids', 'index': 0, 'shape': array([  1, 510], dtype=int32), 'shape_signature': array([  1, 510], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'ref_s', 'index': 1, 'shape': array([  1, 256], dtype=int32), 'shape_signature': array([  1, 256], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'speed', 'index': 2, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([1, 1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_

I0000 00:00:1764488553.382766 3763878 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1693 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:3b:00.0, compute capability: 8.6


In [4]:
input_details = interpreter.get_input_details()
target_shapes = [
    [1, 78],
    [1, 256],
    [1, 1],
]
print(f"{input_details=}")
interpreter.resize_tensor_input(0, target_shapes[0])

interpreter.allocate_tensors()



input_details=[{'name': 'input_ids', 'index': 0, 'shape': array([  1, 510], dtype=int32), 'shape_signature': array([  1, 510], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'ref_s', 'index': 1, 'shape': array([  1, 256], dtype=int32), 'shape_signature': array([  1, 256], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'speed', 'index': 2, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([1, 1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0},

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


RuntimeError: XNNPack delegate failed to reshape runtimeNode number 3317 (TfLiteXNNPackDelegate) failed to prepare.

In [5]:
tf.lite.experimental.Analyzer.analyze("../kokoro.tflite", gpu_compatibility=True)

=== ../kokoro.tflite ===

Your TFLite model has '25' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the GATHER op takes
tensor #565 and tensor #0 as input and produces tensor #602 as output.

Subgraph#0 main(T#0, T#1, T#2) -> [T#4052]
  Op#0 GATHER(T#565, T#0) -> [T#602]
  Op#1 LESS(T#0, T#365[0]) -> [T#603]
  Op#2 ADD(T#0, T#566[178]) -> [T#604]
  Op#3 SELECT(T#603, T#604, T#0) -> [T#605]
  Op#4 GATHER(T#381, T#605) -> [T#606]
  Op#5 RESHAPE(T#606, T#147[1, 1, 510, 512]) -> [T#607]
  Op#6 CONV_2D(T#607, T#247, T#66) -> [T#608]
  Op#7 RESHAPE(T#608, T#146[1, 510, 512]) -> [T#609]
  Op#8 MEAN(T#609, T#408[-1]) -> [T#610]
  Op#9 NEG(T#610) -> [T#611]
  Op#10 SQUARED_DIFFERENCE(T#609, T#610) -> [T#612]
  Op#11 MEAN(T#612, T#408[-1]) -> [T#613]
  Op#12 ADD(T#613, T#593) -> [T#614]
  Op#13 RSQRT(T#614) -> [T#615]
  Op#14 MUL(T#615, T#335) -> [T#616]
  Op#15 MUL(T#611, T#616) -> [T#617]
  Op#16 ADD(T#617, T#334) -> [T#618]
  Op#1